In [1]:
from netam import framework, models
from netam.common import pick_device

In [2]:
# shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
shmoof_data_path = "/Users/matsen/data/shmoof_pcp_2023-11-30_MASKED.csv"
train_df, val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname="51") # , sample_count=5000)

In [3]:
kmer_length = 3
site_count = 500

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, site_count=site_count)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, site_count=site_count)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 2677 training examples and 2323 validation examples


In [4]:
# [I 2023-12-03 13:38:46,435] Trial 37 finished with value: 0.0603752824113363 and parameters: {'kernel_size': 19, 'dropout': 0.0, 'embedding_dim': 12, 'filter_count': 12}. Best is trial 37 with value: 0.0603752824113363.

cat_params = {
    "l2_regularization_coeff": [1e-6, 1e-5, 1e-4],
}
int_params = {
}
float_params = {
}
log_float_params = {
}
# Note that if anything appears below and above, the above gets priority.
fixed_hyperparams = {
    "kmer_length": kmer_length,
    "embedding_dim": 14,
    "filter_count": 25,
    "kernel_size": 15,
    "dropout_prob": 0.1,
    "batch_size": 1024,
    "learning_rate": 0.1,
    "min_learning_rate": 1e-3, # early stopping!
    "l2_regularization_coeff": 1e-6,
    "min_parameter_count": 1000,
    "max_parameter_count": 10000,
}
epochs = 30
optuna_steps = 50

hyper_burrito = framework.SHMHyperBurrito(pick_device(), train_dataset, val_dataset, models.CNNModel, epochs=epochs)

hyper_burrito.optuna_optimize(optuna_steps, cat_params, int_params, float_params, log_float_params, fixed_hyperparams)

[I 2023-12-15 05:33:09,772] A new study created in memory with name: no-name-1d0eaba8-57ea-40bd-a083-37c88980d4e6


Using Metal Performance Shaders
hypers: {'kmer_length': 3, 'embedding_dim': 14, 'filter_count': 25, 'kernel_size': 15, 'dropout_prob': 0.1, 'batch_size': 1024, 'learning_rate': 0.1, 'min_learning_rate': 0.001, 'l2_regularization_coeff': 1e-05, 'min_parameter_count': 1000, 'max_parameter_count': 10000}
burrito_hypers: {'batch_size': 1024, 'learning_rate': 0.1, 'min_learning_rate': 0.001, 'l2_regularization_coeff': 1e-05}


Epoch:  10%|█         | 3/30 [00:03<00:28,  1.07s/it, loss_diff=-0.0324, lr=0.1, val_loss=0.09213] 
[W 2023-12-15 05:33:14,411] Trial 0 failed with parameters: {'l2_regularization_coeff': 1e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/Users/matsen/re/netam/netam/framework.py", line 632, in <lambda>
    lambda trial: self.optuna_objective(
  File "/Users/matsen/re/netam/netam/framework.py", line 617, in optuna_objective
    losses = burrito.full_train(epochs=self.epochs)
  File "/Users/matsen/re/netam/netam/framework.py", line 507, in full_train
    return self.train(*args, **kwargs)
  File "/Users/matsen/re/netam/netam/framework.py", line 417, in train
    train_loss = self.process_data_loader(
  File "/Users/matsen/re/netam/netam/framework.py", line 387, in process_data_load

KeyboardInterrupt: 